In [18]:
!pip install prettytable

In [19]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv
from urllib.parse import quote
import os
from prettytable import PrettyTable

In [20]:
load_dotenv(dotenv_path="password.env")

host=os.getenv("host")
port=3306
user=os.getenv("user")
database=os.getenv("name")
password=os.getenv("password")
password_embedded = quote(password)
password_embedded = quote(password)  

conn = pymysql.connect(
    host=host,
    port=3306,
    user=user,
    database=database,
    password=password)

# Connecting to SQL

connection = create_engine(f"mysql+pymysql://{user}:{password_embedded}@{host}:{port}/{database}")

In [3]:
# Reading in CSVs

inventory = pd.read_csv('inventory.csv')
books = pd.read_csv('books.csv')
authors = pd.read_csv('authors.csv')
patrons = pd.read_csv('patrons.csv')
loans = pd.read_csv('loans.csv')
books['isbn'] = books['isbn'].astype(str)
authors['author_id'] = authors['author_id'].astype('int64')
books['author_id'] = books['author_id'].astype('int64')

In [5]:
# To SQL

inventory.to_sql(name="inventory", con=connection, if_exists='replace', index=False)
books.to_sql(name="books", con=connection, if_exists='replace', index=False)
authors.to_sql(name="authors", con=connection, if_exists='replace', index=False)
patrons.to_sql(name="patrons", con=connection, if_exists='replace', index=False)
loans.to_sql(name="loans", con=connection, if_exists='replace', index=False)

50

In [26]:
#Queries

query1 = "SELECT DISTINCT(Books.Title), Books.pub_year FROM Inventory JOIN Books ON Inventory.ISBN = Books.ISBN ORDER BY Books.pub_year LIMIT 5;"
query2 = "SELECT b.Title AS Title, MAX(CASE WHEN i.Condition = 'As New' OR i.Condition = 'good' THEN 1 ELSE 0 END) AS has_new_book, COUNT(i.ISBN) AS TotalCount, SUM(CASE WHEN l.return_date IS NULL THEN 1 ELSE 0 END) AS UnreturnedCount, COUNT(l.book_id) AS loan_count FROM Inventory i LEFT JOIN Books  b  ON i.ISBN = b.ISBN LEFT JOIN Loans l ON i.book_id = l.book_id GROUP BY b.Title ORDER BY loan_count DESC;"
query3 = "SELECT Books.Genre AS Genre, COUNT(Inventory.ISBN) AS book_count, COUNT(DISTINCT(Books.ISBN)) AS title_count FROM Inventory LEFT JOIN Books ON Inventory.ISBN = Books.ISBN GROUP BY Books.Genre;"
query4 = "SELECT Authors.language, COUNT(*) AS number FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN Authors ON Authors.author_id = Books.author_id GROUP BY Authors.language ORDER BY number;"
query5 = "SELECT `condition`, COUNT(*) FROM Inventory GROUP BY `condition`;"
query6 = "SELECT CONCAT(Authors.first_name, ' ', Authors.last_name) AS full_name, COUNT(*) AS number_of_books FROM Inventory JOIN Books ON Inventory.isbn = Books.isbn JOIN authors ON books.author_id = Authors.author_id GROUP BY full_name;"

In [28]:
def df_to_prettytable(df):
    table = PrettyTable()
    table.field_names = df.columns.tolist()
    for _, row in df.iterrows():
        table.add_row(row.tolist())
    return table

In [29]:
result_df1 = pd.read_sql(query1, con=connection)
result_df1

,Title,pub_year
0,Crime and Punishment,1866
1,The Brothers Karamazov,1880
2,The Stranger,1942
3,The Plague,1947
4,Confessions of a Mask,1949


In [27]:
result_df2 = pd.read_sql(query2, con=connection)
result_df2

,Title,has_new_book,TotalCount,UnreturnedCount,loan_count
0,FORTNITE (Official): Battle Royale Survival Guide,1,12,6.0,7
1,Half of a Yellow Sun,1,6,2.0,4
2,The Brothers Karamazov,1,4,0.0,4
3,The Book of Laughter and Forgetting,0,4,1.0,3
4,NW,0,3,0.0,3
5,Interpreter of Maladies,1,4,1.0,3
6,The Unbearable Lightness of Being,0,2,0.0,2
7,The Stranger,0,2,0.0,2
8,1984,1,2,0.0,2
9,The Vegetarian,0,2,1.0,2


In [7]:
result_df3 = pd.read_sql(query3, con=connection)
result_df3

,Title,Count
0,Miracle Creek,1
1,FORTNITE (Official): Battle Royale Survival Guide,6
2,East of Eden,2
3,Meditations,1
4,Crime and Punishment,1
5,Beloved,1
6,The Alchemist,1
7,White Teeth,1
8,Half of a Yellow Sun,2
9,Americanah,2


In [8]:
result_df4 = pd.read_sql(query4, con=connection)
result_df4

,Genre,Book_Count
0,Classic,7
1,Philosophy,11
2,Thriller,2
3,Self-help,12
4,Dystopian,3
5,Historical,7
6,Literary Fiction,3
7,Science Fiction,5
8,Short Stories,3
9,Contemporary,8


In [9]:
result_df5 = pd.read_sql(query5, con=connection)
result_df5

,Genre,Title_Count
0,Classic,3
1,Contemporary,3
2,Dystopian,2
3,Historical,3
4,Historical Fiction,1
5,Literary Fiction,3
6,Magical Realism,1
7,Philosophy,5
8,Science Fiction,4
9,Self-help,3


In [10]:
result_df6 = pd.read_sql(query6, con=connection)
result_df6

,language,number
0,Korean,1
1,Greek,2
2,Spanish,2
3,Japanese,2
4,French,3
5,Portuguese,3
6,Russian,4
7,Czech,5
8,English,41
